In [1]:
import tensorflow as tf
from keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import torch
import seaborn as sns
from sklearn.metrics import confusion_matrix



In [10]:
def alexnet(input_shape, num_classes):
    model = models.Sequential()

    # Layer 1
    model.add(layers.Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(layers.BatchNormalization())

    # Layer 2
    model.add(layers.Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(layers.BatchNormalization())

    # Layer 3
    model.add(layers.Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))

    # Layer 4
    model.add(layers.Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))

    # Layer 5
    model.add(layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [11]:
input_shape = (227, 227, 3)  # Assuming input images are of size 227x227 with 3 channels (RGB)
num_classes = 3  # Number of classes in ImageNet dataset, change accordingly for your task


model = alexnet(input_shape, num_classes)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 27, 27, 96)        0         
 g2D)                                                            
                                                                 
 batch_normalization_3 (Bat  (None, 27, 27, 96)        384       
 chNormalization)                                                
                                                                 
 conv2d_6 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 13, 13, 256)       0         
 g2D)                                                            
                                                      

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, momentum=0.9),
              metrics=['accuracy'])

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [14]:
train_generator = train_datagen.flow_from_directory(
        './Data/train',
        target_size=(227, 227),
        batch_size=32,
        class_mode='categorical')

Found 5144 images belonging to 3 classes.


In [15]:
validation_generator = test_datagen.flow_from_directory(
        './Data/test',
        target_size=(227, 227),
        batch_size=32,
        class_mode='categorical',
        shuffle=True)

Found 1288 images belonging to 3 classes.


In [16]:
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)

Epoch 1/10
160/160 [==============================] - 428s 3s/step - loss: 2.6206 - accuracy: 0.7171 - val_loss: 3.6397 - val_accuracy: 0.3478
Epoch 2/10
160/160 [==============================] - 443s 3s/step - loss: 0.4204 - accuracy: 0.8482 - val_loss: 0.6296 - val_accuracy: 0.7314
Epoch 3/10
160/160 [==============================] - 432s 3s/step - loss: 0.3415 - accuracy: 0.8766 - val_loss: 0.3270 - val_accuracy: 0.8742
Epoch 4/10
160/160 [==============================] - 420s 3s/step - loss: 0.2981 - accuracy: 0.8933 - val_loss: 0.2760 - val_accuracy: 0.8967
Epoch 5/10
160/160 [==============================] - 435s 3s/step - loss: 0.2902 - accuracy: 0.8956 - val_loss: 0.3214 - val_accuracy: 0.8680
Epoch 6/10
160/160 [==============================] - 443s 3s/step - loss: 0.2726 - accuracy: 0.9022 - val_loss: 0.2020 - val_accuracy: 0.9270
Epoch 7/10
160/160 [==============================] - 421s 3s/step - loss: 0.2508 - accuracy: 0.9075 - val_loss: 0.6293 - val_accuracy: 0.7679

In [29]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nTest accuracy:', test_acc)

41/41 - 28s - loss: 0.1665 - accuracy: 0.9425 - 28s/epoch - 691ms/step

Test accuracy: 0.9425466060638428


In [14]:
true_labels = ['covid', 'pneumonia', 'normal']  # True labels of the test set
predicted_labels = ['covid', 'pneumonia', 'normal']  # Predicted labels of the test set

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Define class labels (if available)
class_labels = ['covid', 'pneumonia', 'normal']  # List of class labels

# Calculate TP, TN, FP, FN
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

# Calculate rates
accuracy = (TP + TN) / np.sum(cm)  # Accuracy
precision = TP / (TP + FP) if (TP + FP) > 0 else 0  # Precision
recall = TP / (TP + FN) if (TP + FN) > 0 else 0  # Recall (Sensitivity)
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0  # Specificity
false_positive_rate = FP / (FP + TN) if (FP + TN) > 0 else 0  # False Positive Rate
false_negative_rate = FN / (FN + TP) if (FN + TP) > 0 else 0  # False Negative Rate

print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall (Sensitivity): {recall:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"False Positive Rate: {false_positive_rate:.2f}")
print(f"False Negative Rate: {false_negative_rate:.2f}")

True Positives (TP): 1
True Negatives (TN): 1
False Positives (FP): 0
False Negatives (FN): 0
Accuracy: 0.67
Precision: 1.00
Recall (Sensitivity): 1.00
Specificity: 1.00
False Positive Rate: 0.00
False Negative Rate: 0.00


In [1]:
# Get true labels
true_labels = validation_generator.classes

# Get predicted probabilities
predictions = model.predict(validation_generator)

# Compute ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
num_classes = len(validation_generator.class_indices)
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(true_labels, predictions[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=0.5)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for AlexNet')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

NameError: name 'validation_generator' is not defined

In [47]:
# model.save('./models/alexnet_model.h5')


c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [46]:
# torch.save(alexnet, './models/alexnet_model.pth')
